In [ ]:
class Particle:
    def __init__(self, particleText):
        
        #Get our list of quantities for the particle
        particleInfo = particleText.split(",")
        
        #Define our particle quantities
        self.obj = particleInfo[0]
        self.E = float(particleInfo[1])
        self.pt = float(particleInfo[2])
        self.eta = float(particleInfo[3])
        self.phi = float(particleInfo[4])
        
    def __str__(self):
        return "(obj: " + self.obj + ", E: " + str(self.E) + ", pt: " + str(self.pt) + ", eta: " + str(self.eta) + ", phi: " +str(self.phi) + ")"
    
    def getQuantity(self, quantityType):
        
        if quantityType == "E":
            return self.E
        
        elif quantityType == "pt":
            return self.pt
        
        elif quantityType == "eta":
            return self.eta
        
        elif quantityType == "phi":
            return self.phi

        else:
            print("Error: Invalid quantity type")
            return 0.0
        
    __repr__=__str__

In [ ]:
import numpy as np
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Dense, Dropout
from keras.utils import plot_model
from sklearn.preprocessing import scale, normalize
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import EarlyStopping
from random import shuffle
from sklearn.metrics import roc_curve, auc, mean_squared_error
from keras.losses import KLDivergence as KL

In [ ]:
def checkJets(obj):
    return obj in ["j", "b"]

def checkLeptons(obj):
    return obj in ["e-", "e+", "m-", "m+"]

def checkPhotons(obj):
    return obj=="g

In [ ]:
def makeParticleList(eventInfo):
    
    #total number of datapoints in the line
    length = len(eventInfo)
    
    particleList = []
    
    #Increment over all of the particles in the line
    for i in range(5, length):
        
        #Get the text for the particle
        particleText = eventInfo[i]
        
        #Make sure that it is actually information for the particle
        if particleText != "" and particleText != "\n":
            
            particle = Particle(particleText)
            
            particleList.append(particle)
    
    return particleList

In [ ]:
def makeEvent(line, signal):
    
    eventInfo = line.split(";")
    
    eventID = eventInfo[0]
    processID = eventInfo[1]
    eventWeight = float(eventInfo[2])
    MET = float(eventInfo[3])
    METPhi = float(eventInfo[4])
    
    signal = float(signal)
    
    particleList = makeParticleList(eventInfo)
    
    crossSection = 1.0
    
    event = {
        "eventID" : eventID,
        "processID" : processID,
        "eventWeight" : eventWeight,
        "MET" : MET,
        "METPhi" : METPhi,
        "particleList" : particleList,
        "crossSection" : crossSection,
        "signal" : signal
    }
    
    return event

In [ ]:
def calculateSignalCrossSection(event, length):
    
    eventWeight = event["eventWeight"]
    
    crossSection = eventWeight*length
    
    return crossSection

In [ ]:
def calculateBackgroundCrossSection(length, luminosity):
    
    crossSection = length/luminosity
    
    return crossSection

In [ ]:
def calculateCrossSection(event, length, luminosity, signal):
    if signal:
        return calculateSignalCrossSection(event, length)
    else:
        return calculateBackgroundCrossSection(length, luminosity)

In [ ]:
def makeDataList(filePath, signal, luminosity = 1.0):
    
    file = open(filePath, "r")
    
    dataList = []
    
    for line in file:
        
        event = makeEvent(line, signal)
        
        dataList.append(event)

    file.close()
    
    length = float(len(dataList))
    
    for event in dataList:
        
        event["crossSection"] = calculateCrossSection(event, length, luminosity, signal)
    
    
    return dataList

In [ ]:
def makeParticleVectors(event):
    
    particleVectors = []
    particleList = event["particleList"]
    length = len(particleList)
    
    #jets = []
    #leptons = []
    
    #for particle in particleList:
        #if checkJets(particle.obj):
            #jets.append(particle)
        #elif checkLeptons(particle.obj):
            #leptons.append(particle)
    
    #jetNumber = len(jets)
    #leptonNumber = len(leptons)
    
    #for i in range(0, 6):
        #if i < jetNumber:
            #jet = jets[i]
            #particleVectors.append(jet.pt)
            #particleVectors.append(jet.eta)
            #particleVectors.append(jet.phi)
            #particleVectors.append(jet.E)
        #else:
            #for j in range(0, 4):
                #particleVectors.append(0.0)
    
    #for i in range(0, 2):
        #if i < leptonNumber:
            #lepton = leptons[i]
            #particleVectors.append(lepton.pt)
            #particleVectors.append(lepton.eta)
            #particleVectors.append(lepton.phi)
            #particleVectors.append(lepton.E)
        #else:
            #for j in range(0, 4):
                #particleVectors.append(0.0)
        
    for i in range(0, 8):
        if i < length:
            particle = particleList[i]
            particleVectors.append(particle.pt)
            particleVectors.append(particle.eta)
            particleVectors.append(particle.phi)
            particleVectors.append(particle.E)
        else:
            for n in range(0, 4):
                particleVectors.append(0.0)
    
    return particleVectors

In [ ]:
def scaleData(data):
    
    scaler = MinMaxScaler()
    scaledData = scaler.fit_transform(data)
    return scaledData

In [ ]:
def formatAutoencoderData(dataList):
    
    autoencoderData = []
    
    for event in dataList:
        particleVectors = makeParticleVectors(event)
        
        #tensor = tf.constant([particleVectors[0], particleVectors[1], particleVectors[2], particleVectors[3], particleVectors[4], particleVectors[5], particleVectors[6], particleVectors[7], event["MET"], event["METPhi"]])
        
        autoencoderData.append([particleVectors[0], particleVectors[1], particleVectors[2], particleVectors[3], particleVectors[4], particleVectors[5], particleVectors[6], particleVectors[7], particleVectors[8], particleVectors[9], particleVectors[10], particleVectors[11], particleVectors[12], particleVectors[13], particleVectors[14], particleVectors[15], particleVectors[16], particleVectors[17], particleVectors[18], particleVectors[19], particleVectors[20], particleVectors[21], particleVectors[22], particleVectors[23], particleVectors[24], particleVectors[25], particleVectors[26], particleVectors[27], particleVectors[28], particleVectors[29], particleVectors[30], particleVectors[31], event["MET"], event["METPhi"], event["signal"]])
    
    scaledData = scaleData(autoencoderData)
    print(scaledData)
    
    autoencoderDataframe = pd.DataFrame(data = scaledData, columns = ["Particle1PT", "Particle1Eta", "Particle1Phi", "Particle1E", "Particle2PT", "Particle2Eta", "Particle2Phi", "Particle2E", "Particle3PT", "Particle3Eta", "Particle3Phi", "Particle3E", "Particle4PT", "Particle4Eta", "Particle4Phi", "Particle4E", "Particle5PT", "Particle5Eta", "Particle5Phi", "Particle5E", "Particle6PT", "Particle6Eta", "Particle6Phi", "Particle6E", "Particle7PT", "Particle7Eta", "Particle7Phi", "Particle7E", "Particle8PT", "Particle8Eta", "Particle8Phi", "Particle8E", "MET", "METPhi", "Signal"])
    
    return autoencoderDataframe

In [ ]:
def calculateAutoencoderError(X_Test, autoencoder):
    
    distortedData = autoencoder.predict(X_Test)
    
    X_Test = X_Test.to_numpy()
    y_predict = []
    
    for i in range(0, len(distortedData)):
        testEvent = X_Test[i]
        distortedEvent = distortedData[i]
        predict = 0.0
        for j in range(0, len(distortedEvent)):
            predict += (distortedEvent[j]-testEvent[j])**2
        y_predict.append(predict)
    
    #y_predict = scaleData(np.reshape(y_predict, (-1, 1)))
    
    return y_predict

In [ ]:
def normalizeArea(hist, bin_edges):
    
    bin_size = bin_edges[1] - bin_edges[0]
    integral = sum(hist) * bin_size
    normalizedCount = (1.0/integral) * hist
    
    return normalizedCount

In [ ]:
def plotHistogram(y_Predict, x_Predict, numBins):
    y_hist, bin_edges = np.histogram(y_Predict, bins=numBins)
    normalizedYCount = normalizeArea(y_hist, bin_edges)
    plt.step(bin_edges[1:], normalizedYCount, label="Signal")
    x_hist, bin_edges = np.histogram(x_Predict, bins=numBins)
    normalizedXCount = normalizeArea(x_hist, bin_edges)
    plt.step(bin_edges[1:], normalizedXCount, label="Background")
    plt.legend()
    plt.show()
    plt.close()

In [ ]:
class Loss_Functions:
    def __init__(self, beta):
        self.beta = beta
    
    def custom_loss_function(y_true, y_pred):
        return (1.0-self.beta)*mean_squared_error(y_true, y_pred) + self.beta*KL(y_true, y_pred)

In [ ]:
from keras import backend as K

def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),
                              mean=0., stddev=0.1)
    return z_mean + K.exp(z_log_sigma) * epsilon

def makeVAE(beta, intermediate_dim, anomalyScore, data):
    
    original_dim = data.shape[1]
    latent_dim = 2

    inputs = Input(shape=(original_dim,))
    h = Dense(intermediate_dim, activation='relu')(inputs)
    z_mean = Dense(latent_dim)(h)
    z_log_sigma = Dense(latent_dim)(h)
    
    z = layers.Lambda(sampling)([z_mean, z_log_sigma])
    
    # Create encoder
    encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')

    # Create decoder
    latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
    x = layers.Dense(intermediate_dim, activation='relu')(latent_inputs)
    outputs = layers.Dense(original_dim, activation='sigmoid')(x)
    decoder = keras.Model(latent_inputs, outputs, name='decoder')

    # instantiate VAE model
    outputs = decoder(encoder(inputs)[2])
    vae = keras.Model(inputs, outputs, name='vae_mlp')
    
    functions = LossFunctions(beta)
    custom_Loss_Function = functions.custom_loss_function
    
    vae.compile(optimizer = "adam", loss = custom_loss_function)
    
    return vae

In [ ]:
backgroundDataList = makeDataList("Data/training_files/training_files/chan1/background_chan1_7.79.csv", False, luminosity = 7.79)
signalDataList = makeDataList("Data/training_files/training_files/chan1/glgl1400_neutralino1100_chan1.csv", True)
length = len(signalDataList)
testDataList = signalDataList+backgroundDataList[:length]
trainingDataList = backgroundDataList[length:]

In [ ]:
autoencoderBackgroundData = formatAutoencoderData(trainingDataList)
autoencoderTestData = formatAutoencoderData(testDataList)

In [ ]:
VarNames = ["Particle1PT", "Particle1Eta", "Particle1Phi", "Particle1E", "Particle2PT", "Particle2Eta", "Particle2Phi", "Particle2E", "Particle3PT", "Particle3Eta", "Particle3Phi", "Particle3E", "Particle4PT", "Particle4Eta", "Particle4Phi", "Particle4E", "Particle5PT", "Particle5Eta", "Particle5Phi", "Particle5E", "Particle6PT", "Particle6Eta", "Particle6Phi", "Particle6E", "Particle7PT", "Particle7Eta", "Particle7Phi", "Particle7E", "Particle8PT", "Particle8Eta", "Particle8Phi", "Particle8E", "MET", "METPhi"]
autoencoderBackgroundData = autoencoderBackgroundData[VarNames]
X_Test = autoencoderTestData[VarNames]
y_Test = autoencoderTestData["Signal"]

In [ ]:
input_dim=autoencoderBackgroundData.shape[1]

input_vec = Input(shape =(input_dim,))

encoded = Dense(34, activation='relu')(input_vec)
encoded = Dense(200, activation='tanh')(encoded)
encoded = Dense(200, activation='tanh')(encoded)
encoded = Dense(20, activation='tanh')(encoded)
mean = Dense(34, )
decoded = Dense(10, activation='tanh')(encoded)
decoded = Dense(20, activation='tanh')(decoded)
decoded = Dense(200, activation='tanh')(decoded)
decoded = Dense(200, activation='tanh')(decoded)
decoded = Dense(input_dim, activation='relu')(decoded)

autoencoder=Model(input_vec, decoded)


autoencoder.compile(optimizer="adam",
                    loss="mean_squared_error") 

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.00005, patience=20)

history=autoencoder.fit(autoencoderBackgroundData, autoencoderBackgroundData, epochs=100,
               batch_size=125,
               shuffle='batch',
               validation_split=0.1)

In [ ]:
y_Predict = calculateAutoencoderError(X_Test, autoencoder)

fpr, tpr, _ = roc_curve(y_Test, y_Predict)
                        
roc_auc = auc(fpr, tpr)

plt.plot(fpr,tpr,color='darkorange',label='ROC curve (area = %0.2f)' % roc_auc)
plt.legend(loc="lower right")
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

plt.show()


In [ ]:
backgroundAutoencoderData = formatAutoencoderData(backgroundDataList)[VarNames]
signalAutoencoderData = formatAutoencoderData(signalDataList)[VarNames]
x_Predict = calculateAutoencoderError(backgroundAutoencoderData, autoencoder)
y_Predict = calculateAutoencoderError(signalAutoencoderData, autoencoder)

plotHistogram(y_Predict, x_Predict, 100)

In [ ]:
plt.plot(range(len(history.history["loss"])),history.history["loss"],label="Training Loss")
plt.plot(range(len(history.history["val_loss"])),history.history["val_loss"],label="Validation Loss")
plt.legend()

In [ ]:
distortedData = autoencoder.predict(X_Test)
y_Predict = []
X_Test_numpy = X_Test.to_numpy()
for i in range(0, len(distortedData)):
    X = X_Test_numpy[i]
    Y = distortedData[i]
    predict = mean_squared_error(X, Y)
    y_Predict.append(predict)
    

fpr, tpr, _ = roc_curve(y_Test, y_Predict)
                        
roc_auc = auc(fpr, tpr)

plt.plot(fpr,tpr,color='darkorange',label='ROC curve (area = %0.2f)' % roc_auc)
plt.legend(loc="lower right")
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

plt.show()